In [ ]:
import pandas as pd
import numpy as np
import random
import gc
from os import listdir
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from keras.optimizers import SGD, RMSprop
import pickle
from sklearn.externals import joblib
from sklearn import tree
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import train_test_split, cross_val_score

# Note types
- 0: nothing
- 1: step
- 2: hold start
- 3: hold/roll end
- 4: roll start
- M: mine

# Classes
- 0: nothing
- 1: one note
- 2: two notes
- 3: three or four notes
- 4: hold start
- 5: roll start
- 6: mine

In [3]:
samples_back_included = 8
num_classes = 7
num_features = 40
num_features_total = (num_features * samples_back_included) + 4
save_files = listdir('data')

def get_features_for_index(beat_features, notes, index):
    if index < 0:
        return [0] * num_features
    return beat_features[index]

def get_class_for_index(notes, index):
    if index < 0:
        return [1, 0, 0, 0, 0, 0, 0]
    row = notes[index][0]
    (steps, holds, rolls, mines) = [row.count(char) for char in ['1', '2', '4', 'M']]
    steps += (holds + rolls)
    return [int(i) for i in [steps == 0 and mines == 0, steps == 1, steps == 2, steps > 2, holds > 0, rolls > 0, mines > 0]]
    
importance_rankings = [48, 24, 12, 16, 6, 8, 3, 4, 2, 1]
def get_beat_importance(index):
    for i in range(len(importance_rankings)):
        if index % importance_rankings[i] == 0:
            return i

def get_features_for_song(key, is_full):
    X = []
    y = []
    if '{0}_beat_features.csv'.format(key) in save_files and '{0}_notes.csv'.format(key) in save_files:
        beat_features_rotated = pd.read_csv('data/{0}_beat_features.csv'.format(key)).values
        notes = pd.read_csv('data/{0}_notes.csv'.format(key), converters={'0': lambda x: str(x)}).values
        beat_features = np.flipud(np.rot90(np.array(beat_features_rotated)))
        num_notes = min(len(notes), len(beat_features))
        for i in range(num_notes):
            row_y = get_class_for_index(notes, i)
            if is_full or (not (row_y == 0 and random.randint(0, 5) != 0)):
                features = [feature for j in range(samples_back_included) for feature in get_features_for_index(beat_features, notes, i - j)]
                features.append(i % 48)
                features.append(get_beat_importance(i))
                features.append(i / 48)
                features.append(num_notes - i / 48)
                X.append(features)
                y.append(row_y)
    return np.array(X), np.array(y)

# Total 243 songs
def build_training_data(is_full = False):
    X = []
    y = []
    songs_to_use = pd.read_csv('data/songs_to_use.csv').values
    for song_data in songs_to_use:
        song_X, song_y = get_features_for_song(song_data[0], is_full)
        X.extend(song_X)
        y.extend(song_y)
    return np.array(X), np.array(y)

In [4]:
X, y = build_training_data(True)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X = None
y = None
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)

In [6]:
y_train = np.array(list(map(lambda one_hot: np.argmax(one_hot), y_train)))
y_test = np.array(list(map(lambda one_hot: np.argmax(one_hot), y_test)))

In [7]:
class_weight = {
    0: 1,
    1: 2,
    2: 4,
    3: 8,
    4: 4,
    5: 4,
    6: 6
}

In [19]:
gb_clf_3 = GradientBoostingClassifier(random_state=0, learning_rate=0.1, n_estimators=25, max_depth=7, subsample=0.85, max_features=200, verbose=True)
gb_clf_3.fit(X_train, y_train)
print (gb_clf_3.score(X_train, y_train))
print (gb_clf_3.score(X_test, y_test))

      Iter       Train Loss      OOB Improve   Remaining Time 
         1      399294.5141       11204.2663          208.25m
         2      350213.9504        8651.8985          199.77m
         3      311856.7528        6767.5352          194.09m
         4      280231.3350        5511.9288          186.26m
         5      254790.4896        4455.9942          177.81m
         6      233494.7307        3709.8141          168.96m
         7      216119.2052        3044.4662          160.66m
         8      201535.0889        2544.9984          176.12m
         9      189173.7304        2116.2806          177.41m
        10      178688.6498        1801.7034          164.06m
        20      129603.6066         383.2786           50.70m
0.89590280492
0.891245956442


In [ ]:
gc.collect()

0

In [ ]:
gb_clf_2 = GradientBoostingClassifier(random_state=0, learning_rate=0.12, n_estimators=25, max_depth=7, subsample=0.85, max_features=200, verbose=True)
gb_clf_2.fit(X_train, y_train)
print (gb_clf_3.score(X_train, y_train))
print (gb_clf_3.score(X_test, y_test))

      Iter       Train Loss      OOB Improve   Remaining Time 
         1      387538.9840       13267.1013          218.10m
         2      332555.1685        9691.3847          206.56m
         3      291095.2093        7325.7545          196.11m
         4      258249.6228        5716.7189          187.63m
         5      232894.3934        4430.3263          178.44m
         6      212088.4601        3606.7338          170.08m
         7      195587.1771        2884.6112          162.25m
         8      182314.6799        2302.9870          165.94m


In [ ]:
gc.collect()

In [15]:
gb_clf_4 = GradientBoostingClassifier(random_state=0, learning_rate=0.07, n_estimators=25, max_depth=7, subsample=0.85, max_features=200, verbose=True)
gb_clf_4.fit(X_train, y_train)
print (gb_clf_4.score(X_train, y_train))
print (gb_clf_4.score(X_test, y_test))

      Iter       Train Loss      OOB Improve   Remaining Time 
         1      417480.7223        8011.1417          207.67m
         2      379675.4871        6674.1207          201.71m
         3      347912.6138        5610.6913          192.61m
         4      320268.2414        4823.6558          184.10m
         5      296657.8593        4138.0421          175.34m
         6      276117.0112        3590.8764          166.64m
         7      258363.1929        3124.0188          159.28m
         8      242806.8004        2728.8316          150.04m
         9      229147.0935        2364.2905          141.50m
        10      217046.2776        2106.1140          133.08m
        20      150919.6633         632.9646           45.17m
0.893491222672
0.889848901898


In [ ]:
gc.collect()

In [ ]:
rf_clf = RandomForestClassifier(n_estimators = 100, verbose=True)
rf_clf.fit(X_train, y_train)
print (rf_clf.score(X_train, y_train))
print (rf_clf.score(X_test, y_test))
# 0.87058119132163025

In [ ]:
rf_clf = RandomForestClassifier(n_estimators = 100, max_features=200, verbose=True)
rf_clf.fit(X_train, y_train)
print (rf_clf.score(X_train, y_train))
print (rf_clf.score(X_test, y_test))
# 0.87058119132163025

In [ ]:
rf_clf = RandomForestClassifier(n_estimators = 100, max_depth=10, verbose=True)
rf_clf.fit(X_train, y_train)
print (rf_clf.score(X_train, y_train))
print (rf_clf.score(X_test, y_test))
# 0.87058119132163025

In [ ]:
rf_clf = RandomForestClassifier(n_estimators = 100, min_samples_leaf=2, verbose=True)
rf_clf.fit(X_train, y_train)
print (rf_clf.score(X_train, y_train))
print (rf_clf.score(X_test, y_test))
# 0.87058119132163025

In [ ]:
gc.collect()

In [ ]:
joblib.dump(clf, 'filename.pkl')
clf = joblib.load('filename.pkl') 

##### sgd_clf = SGDClassifier(loss="log", verbose=True)
sgd_clf.fit(X_train, y_train)
print (sgd_clf.score(X_train, y_train))
print (sgd_clf.score(X_test, y_test))
# 0.849

In [ ]:
def train_and_test(beat_feature_model):
    print ('Train Error')
    print (beat_feature_model.evaluate(X_train, y_train, batch_size=64))
    print ('Test Error')
    print (beat_feature_model.evaluate(X_test, y_test, batch_size=64))
    print ('\n')

In [ ]:
model = Sequential()

model.add(Dense(512, input_shape=(324,)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

model.fit(X, y, nb_epoch=50, batch_size=64, verbose=1)
model.save('models/song_class_model.h5')
model = None
gc.collect()

In [ ]:
model = Sequential()

model.add(Dense(512, input_shape=(324,)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

model.fit(X, y, nb_epoch=50, batch_size=64, verbose=1, class_weight=class_weight)
model.save('models/song_class_model_weighted.h5')
model = None
gc.collect()